<a href="https://colab.research.google.com/github/sanishrai/MNIST_ConvolutionNN/blob/main/3A_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 use the following CNN as our classifier:  5×5  convolution ->  2×2  max pool ->  5×5  convolution ->  2×2  max pool -> fully connected to  ℝ256  -> fully connected to  ℝ10  (prediction). ReLU activation functions will be used to impose non-linearities

 Formula to compute output size: https://www.youtube.com/watch?v=pDdP0TFzsoQ&t=450s
 
 (W-F+2P)/S+1
 Eg: 5*5 input, 3*3 filter, padding=0, stride=1
 (5-3+0)/1+1=
 2/1+1=3
 3*3

In [6]:
import torch.nn as nn
class MNIST_CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,32,kernel_size=5,padding=2)
    self.conv2 = nn.Conv2d(32,64,kernel_size=5, padding=2)
    self.fc1 = nn.Linear(7*7*64,256)#out put after maxpool with kernel 2
    self.fc2 = nn.Linear(256,10)

  def forward(self,x):
    #conv layer 1
    x  = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x,kernel_size=2)

    #conv layer 2
    x= self.conv2(x)
    x=F.relu(x)
    x=F.max_pool2d(x,kernel_size=2)
    
    #fully connected layer
    x=x.view(-1,7*7*64)
    x=self.fc1(x)
    x=F.relu(x)

    #fc layer 2
    x=self.fc2(x)
    return x

  

In [15]:
#using cpu for testing
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm,trange
import time

#load the data
mnist_train = datasets.MNIST(root="./datasets",train=True,transform=transforms.ToTensor(),download=True)
mnist_test = datasets.MNIST(root="./datasets",train=False,transform=transforms.ToTensor(),download=True)
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=100,shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100,shuffle=False)

t0 = time.time()#to check time
#Training
model = MNIST_CNN()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim. Adam(model.parameters(),lr=0.01)
#training
for epoch in trange(3):
  for images, labels in tqdm(train_loader):
    optimizer.zero_grad()
    x=images
    y=model(x)
    loss = criterion(y,labels)
    loss.backward()
    optimizer.step()

torch.cuda.current_stream().synchronize()
t1 = time.time()
print("Cpu time: {}".format(t1-t0))




Cpu time: 307.3415310382843


In [16]:

#testing
## Testing
correct = 0
total = len(mnist_test)

with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(test_loader):
        # Forward pass
        x = images  # <---- change here 
        y = model(x)

        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())

print('Test accuracy: {}'.format(correct/total))


Test accuracy: 0.9812999963760376


In [2]:
import torch
#using gpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [13]:
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm,trange
import time

#load the data
mnist_train = datasets.MNIST(root="./datasets",train=True,transform=transforms.ToTensor(),download=True)
mnist_test = datasets.MNIST(root="./datasets",train=False,transform=transforms.ToTensor(),download=True)
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=100,shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100,shuffle=False)

t0 = time.time()#to check time
## Training
# Instantiate model  
model = MNIST_CNN().to(device)  # <---- change here

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # <---- change here

# Iterate through train set minibatchs 
for epoch in trange(3):  # <---- change here
    for images, labels in tqdm(train_loader):
        # Zero out the gradients
        optimizer.zero_grad()
        labels = labels.to(device)

        # Forward pass
        x = images.to(device)  # <---- change here 
        y = model(x).to(device)
        loss = criterion(y, labels)
        # Backward pass
        loss.backward()
        optimizer.step()

torch.cuda.current_stream().synchronize()
t1 = time.time()   
print("With GPU time:{}".format(t1-t0))




With GPU time:17.62771987915039


In [14]:
import torch
#using gpu for testing
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
with torch.autograd.profiler.profile() as prof:
    ## Testing
    correct = 0
    total = len(mnist_test)

    with torch.no_grad():
        # Iterate through test set minibatchs 
        for images, labels in tqdm(test_loader):
            labels = labels.to(device)
            # Forward pass
            x = images.to(device)  # <---- change here 
            y = model(x).to(device)

            predictions = torch.argmax(y, dim=1)
            correct += torch.sum((predictions == labels).float())

    print('Test accuracy: {}'.format(correct/total))
print(prof)

Using cuda device



Test accuracy: 0.9904999732971191
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                            aten::empty         0.00%      15.779us         0.00%      15.779us      15.779us             1  
                                          aten::random_         0.00%      44.117us         0.00%      44.117us      44.117us             1  
                                aten::is_floating_point         0.00%       1.319us         0.00%       1.319us       1.319us             1  
                                             aten::item         0.01%      80.926us         0.01%      88.547us  